In [ ]:
from airfoil import Airfoil
from airfoil.util.shapely_helpers import plot_shapely_directional
import numpy as np

In [ ]:
def intro_path(coords:np.ndarray, padding:float = 10) -> np.ndarray:
    coords = np.roll(coords,-(coords[:,0]+coords[:,1]).argmax()-1, axis=0)[::-1]
    start = coords[0]
    bottom_left = coords.min(axis=0)
    top_right = coords.max(axis=0)
    width, height = top_right-bottom_left
    
    coords = np.vstack([
        [
            bottom_left - np.array([padding,padding]),
            bottom_left+np.array([-padding,height+padding]),
            top_right + np.array([padding,padding]),
            start + np.array([padding,0])
        ],
        coords,
        [
            coords[-1]+np.array([padding,0])
        ]
    ])
    coords = coords - coords.min(axis=0)
    return coords

In [ ]:
af = Airfoil.from_naca_designation("NACA 2414", 100,50)
ax,_=af.plot()

In [ ]:
import numpy as np
def zigzag(minx,miny,maxx, maxy, peaks:int=20):
    x = np.linspace(0, 1, peaks)
    y = np.zeros_like(x)
    y[::2]=1
    rx = (maxx-minx)*x+minx
    ry = (maxy-miny)*y+miny
    return np.vstack([rx,ry]).transpose()

In [ ]:
from shapely import box, intersection, difference, union, LineString, Polygon
afp = af.polygon()
afpb = afp.bounds
trussed = union(
    intersection(
        afp,
        LineString(zigzag(*afpb,10)).buffer(2)
    ),
    difference(
        afp,
        intersection(
            afp.buffer(-2),
            box(15,-300,80,300)
        )
    )
)
top    = trussed.intersection(box(-500,0,500,500)).exterior
bottom = trussed.intersection(box(-500,-500,500,0)).exterior
#plot_line(bottom)

In [ ]:
ax = plot_shapely_directional([top, bottom])
ax.figure.set_size_inches(20,10)